In [1]:
import os
from athina.evals import (
    RagasAnswerRelevancy,
    RagasContextPrecision,
    RagasFaithfulness,
    RagasContextRecall,
    RagasAnswerSemanticSimilarity,
    RagasAnswerCorrectness,
    RagasHarmfulness,
    RagasMaliciousness,
    RagasCoherence,
    RagasConciseness
)
from athina.runner.run import EvalRunner
from athina.loaders import Loader
from athina.keys import AthinaApiKey, OpenAiApiKey
import pandas as pd

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.core import download_loader


from dotenv import load_dotenv
load_dotenv()

OpenAiApiKey.set_key(os.getenv('OPENAI_API_KEY'))
AthinaApiKey.set_key(os.getenv('ATHINA_API_KEY'))

/Users/akshat_g/athina/repos/athina-evals/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/akshat_g/athina/repos/athina-evals/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# create a llamaindex query engine
WikipediaReader = download_loader("WikipediaReader")
loader = WikipediaReader()
documents = loader.load_data(pages=['Berlin'])
vector_index = VectorStoreIndex.from_documents(
    documents, service_context=ServiceContext.from_defaults(chunk_size=512)
)

query_engine = vector_index.as_query_engine()

/var/folders/0b/_h4f9ghn63nfj95jmn8xwvc40000gn/T/ipykernel_82610/2851782463.py:2: DeprecationWarning: Call to deprecated function (or staticmethod) download_loader. (`download_loader()` is deprecated. Please install tool using pip install directly instead.)
  WikipediaReader = download_loader("WikipediaReader")



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
/var/folders/0b/_h4f9ghn63nfj95jmn8xwvc40000gn/T/ipykernel_82610/2851782463.py:6: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  documents, service_context=ServiceContext.from_defaults(chunk_size=512)


In [3]:
raw_data_llama_index = [
    {
        "query": "Where is Berlin?",
        "expected_response": "Berlin is the capital city of Germany"
    },
    {
        "query": "What is the main cuisine of Rome?",
        "expected_response": "Pasta dish with a sauce made with egg yolks"
    },
]

In [4]:
llama_index_dataset = Loader().load_from_llama_index(raw_data_llama_index, query_engine)
pd.DataFrame(llama_index_dataset)

,query,context,response,expected_response
0,Where is Berlin?,[Berlin is the capital and largest city of Ger...,Berlin is located in northeastern Germany.,Berlin is the capital city of Germany
1,What is the main cuisine of Rome?,[There are seven symphony orchestras in Berlin...,Berlin is well known for its offerings of vege...,Pasta dish with a sauce made with egg yolks


In [5]:
eval_model = "gpt-3.5-turbo"
eval_suite = [
    RagasAnswerCorrectness(model=eval_model, api_key=os.getenv('OPENAI_API_KEY'), provider="openai"),
    RagasFaithfulness(model=eval_model, api_key=os.getenv('OPENAI_API_KEY'), provider="openai"),
    RagasAnswerRelevancy(model=eval_model, api_key=os.getenv('OPENAI_API_KEY'), provider="openai"),
]

# Run the evaluation suite
batch_eval_result = EvalRunner.run_suite(
    evals=eval_suite,
    data=llama_index_dataset,
    max_parallel_evals=1,   # If you increase this, you may run into rate limits
)

pd.DataFrame(batch_eval_result)

TypeError: __init__() missing 3 required positional arguments: 'api_key', 'model', and 'provider'